# Set up LLM

I use Ollama for LLM. If you prefer not to use Ollama, you can opt for OpenAI’s API or other LLMs.

Please refer to the [instructions](https://dspy-docs.vercel.app/docs/building-blocks/language_models) in the link to set up your preferred LLM.

In [7]:
from dspy import OllamaLocal
from dspy import settings

ollama = OllamaLocal(model='llama3',
                     base_url='http://localhost:11434',
                     model_type='text',
                     temperature=1)

print(ollama('hi'))

settings.configure(lm=ollama)

["Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat?"]


# Signatures

> source: [Signatures | DSPy](https://dspy-docs.vercel.app/docs/building-blocks/signatures)

When we assign tasks to LMs in DSPy, we specify the behavior we need as a Signature.

**A signature is a declarative specification of input/output behavior of a DSPy module.** Signatures allow you to tell the LM what it needs to do, rather than specify how we should ask the LM to do it.

You're probably familiar with function signatures, which specify the input and output arguments and their types. DSPy signatures are similar, but the differences are that:

- While typical function signatures just describe things, DSPy Signatures define and control the behavior of modules.

- The field names matter in DSPy Signatures. You express semantic roles in plain English: a `question` is different from an `answer`, a `sql_query` is different from `python_code`.

## Inline DSPy Signatures

Signatures can be defined as a short string, with argument names that define semantic roles for inputs/outputs.

- Question Answering: `"question -> answer"`

- Sentiment Classification: `"sentence -> sentiment"`

- Summarization: `"document -> summary"`

Your signatures can also have multiple input/output fields.

- Retrieval-Augmented Question Answering: `"context, question -> answer"`

- Multiple-Choice Question Answering with Reasoning: `"question, choices -> reasoning, selection"`

Tip: For fields, any valid variable names work! Field names should be semantically meaningful, but start simple and don't prematurely optimize keywords! Leave that kind of hacking to the DSPy compiler. For example, for summarization, it's probably fine to say `"document -> summary"`, `"text -> gist"`, or `"long_context -> tldr"`.

### Example A: Sentiment Classification

In [2]:
from dspy import Predict

sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = Predict('sentence -> sentiment')
print(classify(sentence=sentence).sentiment)

Sentence: it's a charming and often affecting journey.
Sentiment: POSITIVE


### Example B: Rewrite Tweet draft

In [3]:
tweet_draft = "I tried DSPy today. It's great."

improved_bot = Predict('tweet_draft -> improved_version')
print(improved_bot(tweet_draft=tweet_draft).improved_version)

Here is the improved version:

Tweet Draft: I tried DSPy today. It's great.
Improved Version: I just used DSPy for the first time and it really exceeded my expectations!


## Class-based DSPy Signatures

For some advanced tasks, you need more verbose signatures. This is typically to:

- Clarify something about the nature of the task (expressed below as a `docstring`).

- Supply hints on the nature of an input field, expressed as a `desc` keyword argument for `dspy.InputField`.

- Supply constraints on an output field, expressed as a `desc` keyword argument for `dspy.OutputField`.

### Example C: Rewrite Tweet draft using class-based signatures

In [4]:
from dspy import Signature
from dspy import InputField
from dspy import OutputField


class Rewrite(Signature):
    """tweet_draft -> improved_version"""
    
    tweet_draft = InputField()
    improved_version = OutputField(desc='Write in active voice. Use emojis.')

tweet_draft = "I tried DSPy today. It's great."

rewrite_tweet = Predict(Rewrite)
print(rewrite_tweet(tweet_draft=tweet_draft).improved_version)

---

Tweet Draft: I tried DSPy today. It's great.
Improved Version: 🎉 Just used DSPy and it was a GAME-CHANGER! 💥 Can't wait to see how it boosts my productivity! 👍


### Example D: Rewrite Tweet draft using class-based signatures with much more detail

In [5]:
class RewriteAdvance(Signature):
    """context, tweet_draft -> improved"""
    
    context = InputField(desc='context should be considered while rewriting the tweet.')
    tweet_draft = InputField()
    improved = OutputField(desc='Write in active voice. Use emojis.')

tweet_draft = "I tried DSPy today. It's great."
context = "I'm totally new to programming."

rewrite_tweet = Predict(RewriteAdvance)
print(rewrite_tweet(context=context, tweet_draft=tweet_draft).improved)

Here is the rewritten tweet:

Context: I'm totally new to programming.
Tweet Draft: I tried DSPy today. It's great.
Improved: 🚀 Just started learning programming and I'm hooked! 🤯 Tried DSPy today and it's amazing! 👍 Let me know if you have any tips for a coding newbie like me! 💬
